# Notebook Overview

This notebook performs all data loading, cleaning, merging, and restructuring steps for Study 4.  
Use the links below to jump to each section.

---

## 📥 1. Data Loading

1. **BetweenUs Chat Data**  
   [Jump](#1-loading-betweenus-chat-data)

2. **Keys & Identifier Files**  
   [Jump](#2-loading-keys-and-identifiers)

3. **Qualtrics Survey Data**  
   [Jump](#3-loading-qualtrics-survey-data)  
   - Merging with Keys

4. **Avicenna App Data**  
   [Jump](#4-loading-avicenna-data)  
   - Morning Surveys  
   - Afternoon Surveys  
   - Weekend Surveys  
   - Merging with Qualtrics

---

## 🔧 2. Data Processing & Transformation

5. **Weekend Surveys — Wide to Long**  
   [Jump](#5-weekend-surveys-wide-to-long)

6. **Daily Surveys — Wide to Long**  
   [Jump](#6-daily-surveys-wide-to-long)

7. **Filtering for Minimum Survey Completion**  
   [Jump](#7-filtering-for-minimum-completion)

8. **Data Imputation (deprecated)**  
   [Jump](#8-data-imputation-deprecated)

9. **Final Cleaning & Exporting Outputs**  
   [Jump](#9-final-cleaning-and-saving)



In [8]:
import pandas as pd
pd.set_option('display.max_columns', None)

In [7]:
# General config & paths

from pathlib import Path

# Project root = folder where this notebook lives
PROJECT_ROOT = Path().resolve()

# Data folders (relative paths)
DATA_DIR = PROJECT_ROOT / "data"
CHAT_STATS_DIR = DATA_DIR / "chat_stats"
AVICENNA_DIR = DATA_DIR / "avicenna"
WAVE1_DIR = AVICENNA_DIR / "wave1"
WAVE2_DIR = AVICENNA_DIR / "wave2"
WAVE3_DIR = AVICENNA_DIR / "wave3"
WAVE4_DIR = AVICENNA_DIR / "wave4"
WAVE5_DIR = AVICENNA_DIR / "wave5"
WAVE6_DIR = AVICENNA_DIR / "wave6"
KEYS_DIR = DATA_DIR / "keys"
QUALTRICS_DIR = DATA_DIR / "qualtrics"

print("Project root:", PROJECT_ROOT)
print("Chat stats directory:", CHAT_STATS_DIR)
print("Wave 1 directory:", WAVE1_DIR)
print("Wave 2 directory:", WAVE2_DIR)
print("Wave 3 directory:", WAVE3_DIR)
print("Wave 4 directory:", WAVE4_DIR)
print("Wave 5 directory:", WAVE5_DIR)
print("Wave 6 directory:", WAVE6_DIR)
print("Keys directory:", KEYS_DIR)
print("Qualtrics directory:", QUALTRICS_DIR)

Project root: C:\Users\77197jsc\Study 4
Chat stats directory: C:\Users\77197jsc\Study 4\data\chat_stats
Wave 1 directory: C:\Users\77197jsc\Study 4\data\avicenna\wave1
Wave 2 directory: C:\Users\77197jsc\Study 4\data\avicenna\wave2
Wave 3 directory: C:\Users\77197jsc\Study 4\data\avicenna\wave3
Wave 4 directory: C:\Users\77197jsc\Study 4\data\avicenna\wave4
Wave 5 directory: C:\Users\77197jsc\Study 4\data\avicenna\wave5
Wave 6 directory: C:\Users\77197jsc\Study 4\data\avicenna\wave6
Keys directory: C:\Users\77197jsc\Study 4\data\keys
Qualtrics directory: C:\Users\77197jsc\Study 4\data\qualtrics


In [4]:
#Loading datafile Shortcut
half_completes2 = pd.read_csv(DATA_DIR/"processed"/"half_complete_IDs2.csv")
daily_surveys_filtered2 = pd.read_csv(DATA_DIR/"processed"/"daily_surveys_filtered2.csv")

## 1. Loading Between Us chatdata

In [2]:
import os
import re
import pandas as pd

def load_chat_stats(folder: Path) -> pd.DataFrame:
    """
    Load all chat stats CSVs from a folder, and add week/day columns
    based on filenames like 'W1_2.csv'.
    """
    all_data = []

    for csv_path in folder.glob("*.csv"):
        match = re.search(r'W(\d+)_(\d+)', csv_path.name)
        if not match:
            continue  # skip files that don't match the pattern

        week, day = map(int, match.groups())
        df = pd.read_csv(csv_path)
        df["week"] = week
        df["day"] = day
        all_data.append(df)

    return pd.concat(all_data, ignore_index=True)

chat_stats_all = load_chat_stats(CHAT_STATS_DIR)
chat_stats_all.head()

,session_id,msg_count,min_wc,max_wc,median_wc,mean_wc,se_wc,msg_count_wc_above_median,msg_count_wc_above_three,week,day
0,17970,52,1,13,5.0,5.480769,0.411460,30,43,1,1
1,18557,58,1,13,4.5,4.465517,0.346284,29,41,1,1
2,28371,67,1,13,4.0,4.208955,0.350445,37,45,1,1
3,43876,41,1,15,3.0,3.878049,0.505090,23,23,1,1
4,64938,50,1,13,5.5,5.900000,0.466205,25,43,1,1


In [9]:
from scipy.stats import sem

#Keep chats that are also registered in Avicenna
chat_stats_small = chat_stats_all[chat_stats_all["session_id"].astype(float).isin(daily_surveys_filtered2['Movez ID'])]

#Calculate total WC per user
chat_stats_small['total_wc'] = chat_stats_small['msg_count']*chat_stats_small['mean_wc']

#Print chat statistics
print(chat_stats_small['msg_count_wc_above_three'].mean())
print(chat_stats_small['total_wc'].mean())
print(chat_stats_small['se_wc'].mean())
sem_total_wc = sem(chat_stats_small['total_wc'].dropna())
print(sem_total_wc)


25.78489326765189
185.76518883410347
0.6787316218706492
4.155655906383692


C:\Users\77197jsc\AppData\Local\Temp\ipykernel_9792\1242089467.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  chat_stats_small['total_wc'] = chat_stats_small['msg_count']*chat_stats_small['mean_wc']


In [7]:
chat_stats_small.head()

,session_id,msg_count,min_wc,max_wc,median_wc,mean_wc,se_wc,msg_count_wc_above_median,msg_count_wc_above_three,week,day,total_wc
1,18557,58,1,13,4.5,4.465517,0.346284,29,41,1,1,259.0
2,28371,67,1,13,4.0,4.208955,0.350445,37,45,1,1,282.0
3,43876,41,1,15,3.0,3.878049,0.505090,23,23,1,1,159.0
4,64938,50,1,13,5.5,5.900000,0.466205,25,43,1,1,295.0
5,97077,143,1,20,4.0,4.573427,0.253701,81,100,1,1,654.0


## 2. Loading Unique Keys and Identifiers

In [12]:
#Load the Qualtrics Data

qualtrics = pd.read_csv(QUALTRICS_DIR / "Erasmus University Between Us Project_June 2, 2025_06.34.csv")
qualtrics_game = pd.read_csv(QUALTRICS_DIR / "Erasmus University Between Us Project _Game_June 5, 2025_04.03.csv")
qualtrics_game = qualtrics_game[2:]

#Load Identifier keys from the different waves
keys0 = pd.read_csv(KEYS_DIR / "keys_w1.csv")
keys1 = pd.read_csv(KEYS_DIR / "keys_w2.csv")
keys2 = pd.read_csv(KEYS_DIR / "keys_w3.csv")
keys3 = pd.read_csv(KEYS_DIR / "keys_w4.csv")
keys4 = pd.read_csv(KEYS_DIR / "keys_w5.csv")
keys5 = pd.read_csv(KEYS_DIR / "keys_w6.csv")
key5 = keys5.rename(columns = {"Avicenna ID": "ID"})
keys_all = pd.concat([keys0, keys1, keys2, keys3, keys4, keys5], ignore_index=True)
keys_all_unique = keys_all[['ID', 'Movez ID', 'Norstat ID']].drop_duplicates(subset='ID')

keys = pd.read_csv(KEYS_DIR / "full_responses_final.csv")
keys["User ID"] = keys["User ID"].astype(float)

#Manual error correction
keys.loc[keys["User ID"] == 108518, "condition"] = 2
keys_all[keys_all["ID"] == 108626]
keys_all = keys_all.drop(162)


print(keys["condition"].isna().sum())
print(keys["User ID"].nunique())

0
779


In [14]:
# Check for any User IDs in `keys` that do not appear in `keys_all`

keys["User ID"] = pd.to_numeric(keys["User ID"], errors="coerce")
keys_all["ID"] = pd.to_numeric(keys_all["ID"], errors="coerce")

unmatched = keys[~keys["User ID"].isin(keys_all["ID"])]

if unmatched.empty:
    print("✓ All user IDs in `keys` have a matching entry in `keys_all`.")
else:
    print("Unmatched user IDs found:")
    display(unmatched)

✓ All user IDs in `keys` have a matching entry in `keys_all`.


In [15]:
#Creating a single keys file that contains all identifiers and all survey completion rates in one df

keys = keys.drop(columns="Norstat ID")

# Step 1: Initial merge on User ID <-> ID
keys_all = keys.merge(
    keys_all[['ID', 'Movez ID', 'Norstat ID']],
    how='left',
    left_on='User ID',
    right_on='ID'
)

In [16]:
keys_all.head()

,Unnamed: 0.1,Unnamed: 0,User ID,Baseline,Day 1,Day 2,Day 3,Day 4,Day 5,Weekend 1,Day 6,Day 7,Day 8,Day 9,Day 10,Weekend 2,Norstat ID_x,condition,ID,Movez ID,Norstat ID
0,0,0,104582.0,1.0,1,1,1,1,1,1.0,1,1,1,1,1,1.0,5nJ4ApbP4Ne2G3kx,2.0,104582.0,43876.0,NaN
1,1,1,104584.0,1.0,1,1,1,1,1,1.0,1,1,1,1,1,1.0,Vw142r07XqPAaNPX,3.0,104584.0,64938.0,NaN
2,2,2,104605.0,1.0,1,1,1,1,1,1.0,1,1,1,1,1,1.0,yNedoa0VX45AMEjw,5.0,104605.0,97077.0,NaN
3,3,3,104608.0,1.0,1,1,1,1,1,1.0,1,1,1,1,1,1.0,BdVOAGVQVXE968JL,5.0,104608.0,18557.0,NaN
4,4,4,104702.0,1.0,1,1,1,1,1,1.0,1,1,0,1,0,1.0,NaBzAB8D4Qw2YyVM,4.0,104702.0,28371.0,NaN


In [17]:
#Checking for NaN values in our created keys file

print(keys["User ID"].isna().sum())
#keys_all["Movez ID"] = keys_all["Movez ID"].fillna(keys_all["last_name_clean"])
print(keys_all["Movez ID"].isna().sum())
#keys_all["Movez ID"] = keys_all["Movez ID"].fillna(keys_all["Movez ID_y"])
print(keys_all["Movez ID"].isna().sum())

keys_all["Norstat ID_x"] = keys_all["Norstat ID_x"].fillna(keys_all["Norstat ID"])
print(keys_all["Norstat ID_x"].isna().sum())

0
0
0
8


In [18]:
#Creating a Mean message exposure variable for the analysis

keys_all["Message_exposure"] = keys_all[['Baseline', 'Day 1', 'Day 2',
       'Day 3', 'Day 4', 'Day 5', 'Weekend 1', 'Day 6', 'Day 7', 'Day 8',
       'Day 9', 'Day 10']].mean(axis=1)

## 3. Loading Qualtrics Survey Data

In [19]:
#Load and filter survey data for largely complete responses

qualtrics = qualtrics[(qualtrics['Progress'] == "100") & ((qualtrics['Informed_consent_exp'] == "1") | (qualtrics['Q3'] == "1"))]
qualtrics_game["Progress"] = qualtrics_game["Progress"].astype(float)
qualtrics_game = qualtrics_game[qualtrics_game["Progress"] > 66] #We only keep survey data that is at least two-thirds completed

len(qualtrics)

2288

In [20]:
# --- Handle duplicated Qualtrics entries (daily surveys) ---

# How many duplicate Random IDs do we have?
n_dupes = qualtrics["Random ID"].duplicated().sum()
print(f"Initial duplicate Random IDs in qualtrics: {n_dupes}")

# For duplicated IDs, keep the row with the most non-missing data
qualtrics_cleaned = (
    qualtrics
    .assign(non_nan_count=lambda df: df.notna().sum(axis=1))
    .sort_values(by=["Random ID", "non_nan_count"], ascending=[True, False])
    .drop_duplicates(subset="Random ID", keep="first")
    .drop(columns="non_nan_count")
    .reset_index(drop=True)
)

# Sanity check: no duplicates should remain
n_dupes_after = qualtrics_cleaned["Random ID"].duplicated().sum()
print(f"Duplicate Random IDs after cleaning: {n_dupes_after}")
assert n_dupes_after == 0, "There are still duplicate Random IDs after cleaning."


# --- Handle duplicated Qualtrics game entries ---

# Make sure Progress is numeric so "most advanced" can be identified
qualtrics_game["Progress"] = pd.to_numeric(qualtrics_game["Progress"], errors="coerce")

n_dupes_game = qualtrics_game["Child_ID"].duplicated().sum()
print(f"Initial duplicate Child_IDs in qualtrics_game: {n_dupes_game}")

# For duplicated Child_IDs, keep the row with the highest Progress
qualtrics_game_deduped = (
    qualtrics_game
    .sort_values(by=["Child_ID", "Progress"], ascending=[True, False])
    .drop_duplicates(subset="Child_ID", keep="first")
    .reset_index(drop=True)
)

n_dupes_game_after = qualtrics_game_deduped["Child_ID"].duplicated().sum()
print(f"Duplicate Child_IDs after cleaning: {n_dupes_game_after}")
assert n_dupes_game_after == 0, "There are still duplicate Child_IDs after cleaning."


Initial duplicate Random IDs in qualtrics: 37
Duplicate Random IDs after cleaning: 0
Initial duplicate Child_IDs in qualtrics_game: 100
Duplicate Child_IDs after cleaning: 0


In [22]:
# Columns required from parent (baseline) survey
QUALTRICS_COLS = [
    "Mail_child_exp", "Mail_child_ctr", 
    "Country_of_residence", "Country_of_residence_6_TEXT",
    "Dietary_identity", "Dietary_identity_6_TEXT",
    "Children_age_years", "Birthplace_parents",
    "Ethnic_group", "Ethnic_group_5_TEXT",
    "SES_education", "SES_income_UK", "SES_income_ROI",
    "Random ID", "id"
]

# Columns required from children's post-game survey
CHILD_SURVEY_COLS = [
    "Child_ID", "Mail_child", "Imposter", "Game_pleasure",
    "Game_relatedness", "Game_capability", "Sharing_behavior_gam",
    "Game_rating", "Game_feedback", "Sex_1", "Sex_2", "Sex_3",
    "Sex_4", "Gender", "Gender_11_TEXT"
]

#Parents qualtrics surveys
qualtrics = qualtrics_cleaned[QUALTRICS_COLS]

#Children post-game survey
qualtrics_game = qualtrics_game_deduped[CHILD_SURVEY_COLS]

In [27]:
#Lets ranme some columns names
qualtrics_game = qualtrics_game.rename(columns={'Sex_1': 'FFQ_meat', 'Sex_2': 'FFQ_dairy', 'Sex_3': 'FFQ_eggs', 'Sex_4': 'FFQ_fish'})

#And calculate a baseline animal consumption variable for the analysis
qualtrics_game[['FFQ_meat', 'FFQ_dairy', 'FFQ_eggs', 'FFQ_fish']] = qualtrics_game[['FFQ_meat', 'FFQ_dairy', 'FFQ_eggs', 'FFQ_fish']].astype(float)
qualtrics_game["FFQ_average"] = qualtrics_game[['FFQ_meat', 'FFQ_dairy', 'FFQ_eggs', 'FFQ_fish']].mean(axis=1)

In [28]:
qualtrics_game[['FFQ_meat', 'FFQ_dairy', 'FFQ_eggs', 'FFQ_fish', "FFQ_average"]].head()

,FFQ_meat,FFQ_dairy,FFQ_eggs,FFQ_fish,FFQ_average
0,5.0,5.0,5.0,5.0,5.00
1,4.0,5.0,5.0,5.0,4.75
2,4.0,4.0,5.0,2.0,3.75
3,7.0,5.0,8.0,4.0,6.00
4,7.0,7.0,5.0,2.0,5.25


In [29]:
#Lets also make some changes to the parents qualtrics survey

qualtrics = qualtrics[2:]

#Compute SES variable
qualtrics[["SES_education", "SES_income_UK", "SES_income_ROI"]] = (
    qualtrics[["SES_education", "SES_income_UK", "SES_income_ROI"]]
    .apply(pd.to_numeric, errors="coerce")
)

qualtrics["SES"] = (
    qualtrics[["SES_education", "SES_income_UK", "SES_income_ROI"]].astype(float)
    .sum(axis=1, skipna=True) / 2
)

#Create one single mail column for experimental and control groups
qualtrics["Mail_child_exp"] = qualtrics["Mail_child_exp"].fillna(qualtrics["Mail_child_ctr"])

In [31]:
#Lets merge the parents and the childrens' Qualtrics surveys on ID or Mail

# First attempt: merge on Random_ID and Child_ID
merged = qualtrics.merge(qualtrics_game, left_on="Random ID", right_on="Child_ID", how="left")

# Identify unmatched cases
unmatched = merged[merged["Child_ID"].isna()]

# Now attempt to match the unmatched using Mail_child_exp and Mail_child
second_merge = unmatched.drop(columns=qualtrics_game.columns.difference(["Mail_child"]), errors="ignore").merge(
    qualtrics_game, left_on="Mail_child_exp", right_on="Mail_child", how="left"
)

# Combine the successfully matched rows from the first merge (excluding the unmatched)
matched = merged[merged["Child_ID"].notna()]

# Concatenate both results
final_merged = pd.concat([matched, second_merge], ignore_index=True)

print(len(qualtrics))
print(len(final_merged))

print("This worked fine!")

2249
2249
This worked fine!


In [32]:
# -------------------------------------------------------------
# Fix mis-coded Qualtrics scales for game experience variables
# These items were exported with incorrect numeric labels:
# 11→6, 10→5, 9→4, 8→3, 7→2, 1→1
# -------------------------------------------------------------

recode_map = {
    11: 6,
    10: 5,
    9:  4,
    8:  3,
    7:  2,
    1:  1
}

columns_to_recode = ["Game_pleasure", "Game_relatedness", "Game_capability", "Game_rating"]

# Validate columns exist
missing = set(columns_to_recode) - set(final_merged.columns)
assert not missing, f"Missing columns for recoding: {missing}"

# Ensure numeric types before recoding
final_merged[columns_to_recode] = (
    final_merged[columns_to_recode]
    .apply(pd.to_numeric, errors="coerce")
    .replace(recode_map)
)

# Update main dataset
qualtrics = final_merged


In [33]:
final_merged["Game_rating"].value_counts()

Game_rating
5.0    464
4.0    304
3.0    122
2.0     25
1.0      8
Name: count, dtype: int64

In [35]:
# Check rows with a country but missing age
missing_age = qualtrics["Country_of_residence"].notna() & qualtrics["Children_age_years"].isna()

print("Missing age cases:", missing_age.sum())
print("Duplicate Random IDs:", qualtrics["Random ID"].duplicated().sum())


Missing age cases: 0
Duplicate Random IDs: 0


## 3.1. Merging Keys file with Qualtrics data

In [36]:
#Lets prepare the Qualtrics data

# First: ensure qualtrics has no duplicate keys for merging
qualtrics_id_dedup = qualtrics.drop_duplicates(subset='id')
qualtrics_rid_dedup = qualtrics

qualtrics_id_dedup["Random ID"] = pd.to_numeric(qualtrics_id_dedup["Random ID"], errors="coerce")
qualtrics_rid_dedup["Random ID"] = pd.to_numeric(qualtrics_rid_dedup["Random ID"], errors="coerce")

print(qualtrics_id_dedup["Country_of_residence"].isna().sum())

0


C:\Users\77197jsc\AppData\Local\Temp\ipykernel_9792\1637336326.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  qualtrics_id_dedup["Random ID"] = pd.to_numeric(qualtrics_id_dedup["Random ID"], errors="coerce")


In [37]:
#Now we can merge the data with the Keys file Either on ID or Norstat ID

# Step 1: Merge on Movez ID <-> Random ID
step1 = keys_all.merge(
    qualtrics_rid_dedup,
    how='left',
    left_on='Movez ID',
    right_on='Random ID'
)

# Step 2: Identify unmatched from step1
unmatched_step1 = step1[step1['Random ID'].isna()].copy()

# Step 3: Drop overlapping Qualtrics columns
qualtrics_cols = qualtrics.columns.difference(keys_all.columns.union(['id']))
unmatched_step1 = unmatched_step1.drop(columns=qualtrics_cols, errors='ignore')

# Step 4: Merge unmatched based on Norstat ID <-> id
step2 = unmatched_step1.merge(
    qualtrics_id_dedup[['id'] + list(qualtrics_cols)],
    how='left',
    left_on='Norstat ID',
    right_on='id'
)

# Step 5: Combine matched from step1 and step2
qualtrics_data = pd.concat([step1[~step1['Random ID'].isna()], step2], ignore_index=True)

# Final check
assert len(qualtrics_data) == len(keys_all), f"Expected {len(keys_all)} rows, got {len(qualtrics_data)}"


In [38]:
#Lets see if it worked
qualtrics_data.head()

,Unnamed: 0.1,Unnamed: 0,User ID,Baseline,Day 1,Day 2,Day 3,Day 4,Day 5,Weekend 1,Day 6,Day 7,Day 8,Day 9,Day 10,Weekend 2,Norstat ID_x,condition,ID,Movez ID,Norstat ID,Message_exposure,Mail_child_exp,Mail_child_ctr,Country_of_residence,Country_of_residence_6_TEXT,Dietary_identity,Dietary_identity_6_TEXT,Children_age_years,Birthplace_parents,Ethnic_group,Ethnic_group_5_TEXT,SES_education,SES_income_UK,SES_income_ROI,Random ID,id,SES,Child_ID,Mail_child,Imposter,Game_pleasure,Game_relatedness,Game_capability,Sharing_behavior_gam,Game_rating,Game_feedback,FFQ_meat,FFQ_dairy,FFQ_eggs,FFQ_fish,Gender,Gender_11_TEXT,FFQ_average,Mail_child_x,Mail_child_y,id_x,id_y
0,0,0,104582.0,1.0,1,1,1,1,1,1.0,1,1,1,1,1,1.0,5nJ4ApbP4Ne2G3kx,2.0,104582.0,43876.0,NaN,1.000000,romeobenson23@gmail.com,NaN,1,NaN,1,NaN,5,2,NaN,NaN,5.0,5.0,NaN,43876.0,5nJ4ApbP4Ne2G3kx,5.0,43876,romeobenson23@gmail.com,NaN,6.0,6.0,6.0,3,5.0,NaN,5.0,5.0,5.0,5.0,7,NaN,5.00,NaN,NaN,NaN,NaN
1,1,1,104584.0,1.0,1,1,1,1,1,1.0,1,1,1,1,1,1.0,Vw142r07XqPAaNPX,3.0,104584.0,64938.0,NaN,1.000000,gaagoobilo@gmail.com,NaN,1,NaN,1,NaN,3,2,NaN,NaN,4.0,4.0,NaN,64938.0,Vw142r07XqPAaNPX,4.0,64938,gaagoobilo@gmail.com,NaN,6.0,6.0,6.0,6,5.0,I liked the simple user interface. I disliked ...,4.0,4.0,4.0,4.0,7,NaN,4.00,NaN,NaN,NaN,NaN
2,2,2,104605.0,1.0,1,1,1,1,1,1.0,1,1,1,1,1,1.0,yNedoa0VX45AMEjw,5.0,104605.0,97077.0,NaN,1.000000,irishvader@outlook.com,NaN,1,NaN,1,NaN,5,2,NaN,NaN,3.0,2.0,NaN,97077.0,yNedoa0VX45AMEjw,2.5,97077,IrishVader@outlook.com,NaN,6.0,5.0,6.0,NaN,4.0,very easy for imposter (me) i just looked up q...,8.0,2.0,4.0,5.0,7,NaN,4.75,NaN,NaN,NaN,NaN
3,3,3,104608.0,1.0,1,1,1,1,1,1.0,1,1,1,1,1,1.0,BdVOAGVQVXE968JL,5.0,104608.0,18557.0,NaN,1.000000,likohgre@gmail.com,NaN,1,NaN,1,NaN,2,2,NaN,NaN,4.0,4.0,NaN,18557.0,BdVOAGVQVXE968JL,4.0,18557,likohgre@gmail.com,NaN,6.0,6.0,6.0,6,5.0,NaN,4.0,4.0,5.0,4.0,7,NaN,4.25,NaN,NaN,NaN,NaN
4,4,4,104702.0,1.0,1,1,1,1,1,1.0,1,1,0,1,0,1.0,NaBzAB8D4Qw2YyVM,4.0,104702.0,28371.0,NaN,0.833333,Gizmo910@icloud.com,NaN,1,NaN,1,NaN,1,2,NaN,NaN,2.0,6.0,NaN,28371.0,NaBzAB8D4Qw2YyVM,4.0,28371,gizmo910@icloud.com,NaN,6.0,6.0,6.0,"5,6",5.0,i would add more ideas like do you like a cert...,7.0,2.0,4.0,2.0,7,NaN,3.75,NaN,NaN,NaN,NaN


In [39]:
#We can drop duplicated columns that we dont need anymore
qualtrics_data = qualtrics_data.drop(columns=["Mail_child_x", "Mail_child_y", "id_x", "id_y"])

In [41]:
import numpy as np

#Again we need to take care of some false Qualtrics coding
print(qualtrics_data['Dietary_identity'].value_counts(dropna="false"))

qualtrics_data['Dietary_identity'] = qualtrics_data['Dietary_identity'].replace({
    6: np.nan,
    7: np.nan,
    2: 3,
    3: 2
})

Dietary_identity
1    621
3    135
2     15
7      6
6      2
Name: count, dtype: int64


In [42]:
print("Lets check for NaN values:", qualtrics_data['Country_of_residence'].isna().sum())

Lets check for NaN values: 2


## 4. Loading Avicenna Data

### 4.1. Morning surveys

In [43]:
#Lets Load the Avicenna Survey Data now with the Lunch-choice measures

# Reuse the cleaning function
def clean_wave(df):
    # Step 1: Remove rows where Duration == "Expired" if Name is duplicated
    df = df[~((df.duplicated(subset='Name', keep=False)) & (df['Duration (seconds) from scheduled to completion time'] == "Expired"))]
    
    # Step 2: Keep only the last row for remaining duplicates
    df = df.drop_duplicates(subset='Name', keep='last')
    
    return df.reset_index(drop=True)


# We'll start with importing the morning daily surveys for the 10 days (after Wave 5)
d11 = clean_wave(pd.read_csv(WAVE5_DIR / "2.1 Daily Survey.csv"))
d21 = clean_wave(pd.read_csv(WAVE5_DIR / "3.1 Daily Survey.csv"))
d31 = clean_wave(pd.read_csv(WAVE5_DIR / "4.1 Daily Survey.csv"))
d41 = clean_wave(pd.read_csv(WAVE5_DIR / "5.1 Daily Survey.csv"))
d51 = clean_wave(pd.read_csv(WAVE5_DIR / "6.1 Daily Survey.csv"))
d61 = clean_wave(pd.read_csv(WAVE5_DIR / "8.1 Daily Survey.csv"))
d71 = clean_wave(pd.read_csv(WAVE5_DIR / "9.1 Daily Survey.csv"))
d81 = clean_wave(pd.read_csv(WAVE5_DIR / "10.1 Daily Survey.csv"))
d91 = clean_wave(pd.read_csv(WAVE5_DIR / "11.1 Daily Survey.csv"))
d101 = clean_wave(pd.read_csv(WAVE5_DIR / "12.1 Daily Survey.csv"))

# Start with the first DataFrame
morning_surveys = d11

# Merge the others one by one with suffixes to differentiate columns
for i, df in enumerate([d21, d31, d41, d51, d61, d71, d81, d91, d101], start=2):
    suffix = f"_{i}"  # Adds suffix to the columns of each new dataframe
    morning_surveys = pd.merge(morning_surveys, df, on="Name", how="right", suffixes=('', suffix))


print(len(morning_surveys))

477


In [44]:
# Importing the morning daily surveys for the 10 days (after Wave 6)

d11_2 = clean_wave(pd.read_csv(WAVE6_DIR / "2.1. Daily Survey.csv"))
d21_2 = clean_wave(pd.read_csv(WAVE6_DIR / "3.1. Daily Survey.csv"))
d31_2 = clean_wave(pd.read_csv(WAVE6_DIR / "4.1. Daily Survey.csv"))
d41_2 = clean_wave(pd.read_csv(WAVE6_DIR / "5.1. Daily Survey.csv"))
d51_2 = clean_wave(pd.read_csv(WAVE6_DIR / "6.1. Daily Survey.csv"))
d61_2 = clean_wave(pd.read_csv(WAVE6_DIR / "8.1. Daily Survey.csv"))
d71_2 = clean_wave(pd.read_csv(WAVE6_DIR / "9.1. Daily Survey.csv"))
d81_2 = clean_wave(pd.read_csv(WAVE6_DIR / "10.1. Daily Survey.csv"))
d91_2 = clean_wave(pd.read_csv(WAVE6_DIR / "11.1. Daily Survey.csv"))
d101_2 = clean_wave(pd.read_csv(WAVE6_DIR / "12.1. Daily Survey.csv"))

# Start with the first DataFrame
morning_surveys2 = d11_2

# Merge the others one by one with suffixes to differentiate columns
for i, df in enumerate([d21_2, d31_2, d41_2, d51_2, d61_2, d71_2, d81_2, d91_2, d101_2], start=2):
    suffix = f"_{i}"  # Adds suffix to the columns of each new dataframe
    morning_surveys2 = pd.merge(morning_surveys2, df, on="Name", how="right", suffixes=('', suffix))


print(len(morning_surveys2))

509


In [45]:
#Lets make sure the Avicenna Time Columns is formatted correctly

# Step 1: Remove trailing time zone strings (like " BST", " WAT", etc.)
morning_surveys2['Scheduled Time'] = (
    morning_surveys2['Scheduled Time']
    .astype(str)
    .str.replace(r'\s*([A-Z]{2,5}|[+-]?\d{2,4})$', '', regex=True)
)

# Step 2: Convert to datetime
morning_surveys2['Scheduled Time'] = pd.to_datetime(
    morning_surveys2['Scheduled Time'], errors='coerce'
)

# Step 3: Drop timezone info (just in case)
if morning_surveys2['Scheduled Time'].dt.tz is not None:
    morning_surveys2['Scheduled Time'] = morning_surveys2['Scheduled Time'].dt.tz_localize(None)

# Step 4: Filter for May 1st or later (this is the final wave that we will append later on)
morning_surveys2 = morning_surveys2[
    morning_surveys2['Scheduled Time'] >= pd.Timestamp('2025-05-01')
]

# Step 5: Check result
print("Remaining rows from the final wave:", len(morning_surveys2))

Remaining rows from the final wave: 32


In [46]:
morning_surveys2.head(3)

,Name,Device ID,Scheduled Time,Issued Time,Response Time,Duration (seconds) from scheduled to completion time,Duration (seconds) from first response to completion time,Location,Message_liking1,Message_shareability1,Message_dislike_reason1,Message_relevance1_1,Message_relevance1_2,Message_relevance1_3,Session UUID,Device ID_2,Scheduled Time_2,Issued Time_2,Response Time_2,Duration (seconds) from scheduled to completion time_2,Duration (seconds) from first response to completion time_2,Location_2,Message_liking2,Message_shareability2,Message_dislike_reason2,Message_relevance2_1,Message_relevance2_2,Message_relevance2_3,Session UUID_2,Device ID_3,Scheduled Time_3,Issued Time_3,Response Time_3,Duration (seconds) from scheduled to completion time_3,Duration (seconds) from first response to completion time_3,Location_3,Message_liking3,Message_shareability3,Message_dislike_reason3,Message_relevance3_1,Message_relevance3_2,Message_relevance3_3,Session UUID_3,Device ID_4,Scheduled Time_4,Issued Time_4,Response Time_4,Duration (seconds) from scheduled to completion time_4,Duration (seconds) from first response to completion time_4,Location_4,Message_liking4,Message_shareability4,Message_dislike_reason4,Message_relevance4_1,Message_relevance4_2,Message_relevance4_3,Session UUID_4,Device ID_5,Scheduled Time_5,Issued Time_5,Response Time_5,Duration (seconds) from scheduled to completion time_5,Duration (seconds) from first response to completion time_5,Location_5,Message_liking5,Message_shareability5,Message_dislike_reason5,Message_relevance5_1,Message_relevance5_2,Message_relevance5_3,Session UUID_5,Device ID_6,Scheduled Time_6,Issued Time_6,Response Time_6,Duration (seconds) from scheduled to completion time_6,Duration (seconds) from first response to completion time_6,Location_6,Message_liking6,Message_shareability6,Message_dislike_reason6,Message_relevance6_1,Message_relevance6_2,Message_relevance6_3,Session UUID_6,Device ID_7,Scheduled Time_7,Issued Time_7,Response Time_7,Duration (seconds) from scheduled to completion time_7,Duration (seconds) from first response to completion time_7,Location_7,Message_liking7,Message_shareability7,Message_dislike_reason7,Message_relevance7_1,Message_relevance7_2,Message_relevance7_3,Session UUID_7,Device ID_8,Scheduled Time_8,Issued Time_8,Response Time_8,Duration (seconds) from scheduled to completion time_8,Duration (seconds) from first response to completion time_8,Location_8,Message_liking8,Message_shareability8,Message_dislike_reason8,Message_relevance8_1,Message_relevance8_2,Message_relevance8_3,Session UUID_8,Device ID_9,Scheduled Time_9,Issued Time_9,Response Time_9,Duration (seconds) from scheduled to completion time_9,Duration (seconds) from first response to completion time_9,Location_9,Message_liking9,Message_shareability9,Message_dislike_reason9,Message_relevance9_1,Message_relevance9_2,Message_relevance9_3,Session UUID_9,Device ID_10,Scheduled Time_10,Issued Time_10,Response Time_10,Duration (seconds) from scheduled to completion time_10,Duration (seconds) from first response to completion time_10,Location_10,Message_liking10,Message_shareability10,Message_dislike_reason10,Message_relevance10_1,Message_relevance10_2,Message_relevance10_3,Session UUID_10
474,120931,E8CD7F450FF64EFDAA974829AAF449F1,2025-05-26 06:30:00,2025-05-26 06:30:00 WAT,2025-05-26 07:17:22 WAT,2842135,2842,Unknown,3.0,3.0,NaN,7.0,7.0,7.0,0e0f51ca-1d10-4db5-9636-684730c8b679,E8CD7F450FF64EFDAA974829AAF449F1,2025-05-27 06:30:00 WAT,2025-05-27 06:30:00 WAT,2025-05-27 06:41:43 WAT,703138,703,Unknown,3.0,3.0,NaN,7.0,7.0,7.0,6db041a6-02a9-45ed-803e-3fe0566a45dc,E8CD7F450FF64EFDAA974829AAF449F1,2025-05-28 06:30:00 WAT,2025-05-28 06:30:00 WAT,2025-05-28 06:51:11 WAT,1271003,1271,Unknown,3.0,3.0,NaN,7.0,7.0,7.0,39187bff-8565-4941-ba3a-6d62c7c41dcd,E8CD7F450FF64EFDAA974829AAF449F1,2025-05-29 06:30:00 WAT,2025-05-29 06:30:00 WAT,2025-05-29 06:34:48 WAT,288980,289,Unknown,3.0,3.0,NaN,7.0,7.0,7.0,7b7a4539-e4d8-4b0e-a05d-58ccd70e

In [47]:
#Lets add the final wave to the other morning surveys

morning_surveys = pd.concat([morning_surveys, morning_surveys2])
print(len(morning_surveys))

509


In [48]:
# Compute daily relevance scores
for day in range(1, 11):
    cols = [f"Message_relevance{day}_{i}" for i in range(1, 4)]
    morning_surveys[f"Message_relevance{day}"] = morning_surveys[cols].mean(axis=1)

# Minimal check for duplicate names
name_counts = morning_surveys["Name"].value_counts()
dupes = name_counts[name_counts > 1]

print(f"Duplicate Names: {len(dupes)} | Total rows: {len(morning_surveys)}")
print(dupes)


Duplicate Names: 0 | Total rows: 509
Series([], Name: count, dtype: int64)


### 4.2. Afternoon surveys

In [49]:
# Importing the afternoon daily surveys for the 10 days (after Wave 5)
d12 = clean_wave(pd.read_csv(WAVE5_DIR / "2.2 Daily Survey.csv"))
d22 = clean_wave(pd.read_csv(WAVE5_DIR / "3.2 Daily Survey.csv"))
d32 = clean_wave(pd.read_csv(WAVE5_DIR / "4.2 Daily Survey.csv"))
d42 = clean_wave(pd.read_csv(WAVE5_DIR / "5.2 Daily Survey.csv"))
d52 = clean_wave(pd.read_csv(WAVE5_DIR / "6.2 Daily Survey.csv"))
d62 = clean_wave(pd.read_csv(WAVE5_DIR / "8.2 Daily Survey.csv"))
d72 = clean_wave(pd.read_csv(WAVE5_DIR / "9.2 Daily Survey.csv"))
d82 = clean_wave(pd.read_csv(WAVE5_DIR / "10.2 Daily Survey.csv"))
d92 = clean_wave(pd.read_csv(WAVE5_DIR / "11.2 Daily Survey.csv"))
d102 = clean_wave(pd.read_csv(WAVE5_DIR / "12.2 Daily Survey.csv"))

# Start with the first DataFrame
afternoon_surveys = d12

# Merge the others one by one with suffixes to differentiate columns
for i, df in enumerate([d22, d32, d42, d52, d62, d72, d82, d92, d102], start=2):
    suffix = f"_{i}"  # Adds suffix to the columns of each new dataframe
    afternoon_surveys = pd.merge(afternoon_surveys, df, on="Name", how="right", suffixes=('', suffix))

print(len(afternoon_surveys))

640


In [51]:
# Importing the afternoon daily surveys from the final wave

d12_2 = clean_wave(pd.read_csv(WAVE6_DIR / "2.2. Daily Survey.csv"))
d22_2 = clean_wave(pd.read_csv(WAVE6_DIR / "3.2. Daily Survey.csv"))
d32_2 = clean_wave(pd.read_csv(WAVE6_DIR / "4.2. Daily Survey.csv"))
d42_2 = clean_wave(pd.read_csv(WAVE6_DIR / "5.2. Daily Survey.csv"))
d52_2 = clean_wave(pd.read_csv(WAVE6_DIR / "6.2. Daily Survey.csv"))
d62_2 = clean_wave(pd.read_csv(WAVE6_DIR / "8.2. Daily Survey.csv"))
d72_2 = clean_wave(pd.read_csv(WAVE6_DIR / "9.2. Daily Survey.csv"))
d82_2 = clean_wave(pd.read_csv(WAVE6_DIR / "10.2. Daily Survey.csv"))
d92_2 = clean_wave(pd.read_csv(WAVE6_DIR / "11.2. Daily Survey.csv"))
d102_2 = clean_wave(pd.read_csv(WAVE6_DIR / "12.2. Daily Survey.csv"))

# Start with the first DataFrame
afternoon_surveys2 = d12_2

# Merge the others one by one with suffixes to differentiate columns
for i, df in enumerate([d22_2, d32_2, d42_2, d52_2, d62_2, d72_2, d82_2, d92_2, d102_2], start=2):
    suffix = f"_{i}"  # Adds suffix to the columns of each new dataframe
    afternoon_surveys2 = pd.merge(afternoon_surveys2, df, on="Name", how="right", suffixes=('', suffix))

print(len(afternoon_surveys2))

680


In [52]:
#Again lets make sure the Avicenna time column is set correctly, and let's filter for the final wave entries

# Step 1: Remove trailing time zone strings (like " BST", " WAT", etc.)
afternoon_surveys2['Scheduled Time'] = (
    afternoon_surveys2['Scheduled Time']
    .astype(str)
    .str.replace(r'\s*([A-Z]{2,5}|[+-]?\d{2,4})$', '', regex=True)
)

# Step 2: Convert to datetime
afternoon_surveys2['Scheduled Time'] = pd.to_datetime(
    afternoon_surveys2['Scheduled Time'], errors='coerce'
)

# Step 3: Drop timezone info (just in case)
if afternoon_surveys2['Scheduled Time'].dt.tz is not None:
    afternoon_surveys2['Scheduled Time'] = afternoon_surveys2['Scheduled Time'].dt.tz_localize(None)

# Step 4: Filter for May 1st or later
afternoon_surveys2 = afternoon_surveys2[
    afternoon_surveys2['Scheduled Time'] >= pd.Timestamp('2025-05-01')
]

# Step 5: Check result
print("Remaining rows after filter:", len(afternoon_surveys2))

Remaining rows after filter: 42


In [55]:
#Merging the final wave afternoon surveys with the other afternoon surveys
afternoon_surveys = pd.concat([afternoon_surveys, afternoon_surveys2])

print("Total amount:", len(afternoon_surveys))

Total amount: 724


### 4.3. Merging morning and afternoon surveys

In [57]:
#Merging morning and afternoon survez left, so that that the control group people are retained
daily_surveys = afternoon_surveys.merge(morning_surveys, how="left", on="Name")

In [58]:
daily_surveys.head(3)

,Name,Device ID_x,Scheduled Time_x,Issued Time_x,Response Time_x,Duration (seconds) from scheduled to completion time_x,Duration (seconds) from first response to completion time_x,Location_x,Lunch_open1,Lunch_plantbased1,Lunch_availability1,Session UUID_x,Device ID_2_x,Scheduled Time_2_x,Issued Time_2_x,Response Time_2_x,Duration (seconds) from scheduled to completion time_2_x,Duration (seconds) from first response to completion time_2_x,Location_2_x,Lunch_open2,Lunch_plantbased2,Lunch_availability2,Session UUID_2_x,Device ID_3_x,Scheduled Time_3_x,Issued Time_3_x,Response Time_3_x,Duration (seconds) from scheduled to completion time_3_x,Duration (seconds) from first response to completion time_3_x,Location_3_x,Lunch_open3,Lunch_plantbased3,Lunch_availability3,Session UUID_3_x,Device ID_4_x,Scheduled Time_4_x,Issued Time_4_x,Response Time_4_x,Duration (seconds) from scheduled to completion time_4_x,Duration (seconds) from first response to completion time_4_x,Location_4_x,Lunch_open4,Lunch_plantbased4,Lunch_availability4,Session UUID_4_x,Device ID_5_x,Scheduled Time_5_x,Issued Time_5_x,Response Time_5_x,Duration (seconds) from scheduled to completion time_5_x,Duration (seconds) from first response to completion time_5_x,Location_5_x,Lunch_open5,Lunch_plantbased5,Lunch_availability5,Session UUID_5_x,Device ID_6_x,Scheduled Time_6_x,Issued Time_6_x,Response Time_6_x,Duration (seconds) from scheduled to completion time_6_x,Duration (seconds) from first response to completion time_6_x,Location_6_x,Lunch_open6,Lunch_plantbased6,Lunch_availability6,Session UUID_6_x,Device ID_7_x,Scheduled Time_7_x,Issued Time_7_x,Response Time_7_x,Duration (seconds) from scheduled to completion time_7_x,Duration (seconds) from first response to completion time_7_x,Location_7_x,Lunch_open7,Lunch_plantbased7,Lunch_availability7,Session UUID_7_x,Device ID_8_x,Scheduled Time_8_x,Issued Time_8_x,Response Time_8_x,Duration (seconds) from scheduled to completion time_8_x,Duration (seconds) from first response to completion time_8_x,Location_8_x,Lunch_open8,Lunch_plantbased8,Lunch_availability8,Session UUID_8_x,Device ID_9_x,Scheduled Time_9_x,Issued Time_9_x,Response Time_9_x,Duration (seconds) from scheduled to completion time_9_x,Duration (seconds) from first response to completion time_9_x,Location_9_x,Lunch_open9,Lunch_plantbased9,Lunch_availability9,Session UUID_9_x,Device ID_10_x,Scheduled Time_10_x,Issued Time_10_x,Response Time_10_x,Duration (seconds) from scheduled to completion time_10_x,Duration (seconds) from first response to completion time_10_x,Location_10_x,Lunch_open10,Lunch_plantbased10,Lunch_availability10,Session UUID_10_x,Device ID_y,Scheduled Time_y,Issued Time_y,Response Time_y,Duration (seconds) from scheduled to completion time_y,Duration (seconds) from first response to completion time_y,Location_y,Message_liking1,Message_shareability1,Message_dislike_reason1,Message_relevance1_1,Message_relevance1_2,Message_relevance1_3,Session UUID_y,Device ID_2_y,Scheduled Time_2_y,Issued Time_2_y,Response Time_2_y,Duration (seconds) from scheduled to completion time_2_y,Duration (seconds) from first response to completion time_2_y,Location_2_y,Message_liking2,Message_shareability2,Message_dislike_reason2,Message_relevance2_1,Message_relevance2_2,Message_relevance2_3,Session UUID_2_y,Device ID_3_y,Scheduled Time_3_y,Issued Time_3_y,Response Time_3_y,Duration (seconds) from scheduled to completion time_3_y,Duration (seconds) from first response to completion time_3_y,Location_3_y,Message_liking3,Message_shareability3,Message_dislike_reason3,Message_relevance3_1,Message_relevance3_2,Message_relevance3_3,Session UUID_3_y,Device ID_4_y,Scheduled Time_4_y,Issued Time_4_y,Response Time_4_y,Duration (seconds) from scheduled to completion time_4_y,Duration (seconds) from first response to completion time_4_y,Location_4_y,Message_liking4,Message_shareability4,Message_dislike_reason4,Message_relevance4_1,Message_relevance4_2,Message_relevance4_3,Session 

### 4.4 Merging Avicenna surveys with Qualtrics surveys

In [59]:
# Merge daily surveys with Qualtrics keys
daily_surveys["Name"] = pd.to_numeric(daily_surveys["Name"], errors="coerce")

daily_surveys = daily_surveys.merge(
    keys_all,
    left_on="Name",
    right_on="User ID",
    how="left"
)


In [61]:
daily_surveys.head(3)

,Name,Device ID_x,Scheduled Time_x,Issued Time_x,Response Time_x,Duration (seconds) from scheduled to completion time_x,Duration (seconds) from first response to completion time_x,Location_x,Lunch_open1,Lunch_plantbased1,Lunch_availability1,Session UUID_x,Device ID_2_x,Scheduled Time_2_x,Issued Time_2_x,Response Time_2_x,Duration (seconds) from scheduled to completion time_2_x,Duration (seconds) from first response to completion time_2_x,Location_2_x,Lunch_open2,Lunch_plantbased2,Lunch_availability2,Session UUID_2_x,Device ID_3_x,Scheduled Time_3_x,Issued Time_3_x,Response Time_3_x,Duration (seconds) from scheduled to completion time_3_x,Duration (seconds) from first response to completion time_3_x,Location_3_x,Lunch_open3,Lunch_plantbased3,Lunch_availability3,Session UUID_3_x,Device ID_4_x,Scheduled Time_4_x,Issued Time_4_x,Response Time_4_x,Duration (seconds) from scheduled to completion time_4_x,Duration (seconds) from first response to completion time_4_x,Location_4_x,Lunch_open4,Lunch_plantbased4,Lunch_availability4,Session UUID_4_x,Device ID_5_x,Scheduled Time_5_x,Issued Time_5_x,Response Time_5_x,Duration (seconds) from scheduled to completion time_5_x,Duration (seconds) from first response to completion time_5_x,Location_5_x,Lunch_open5,Lunch_plantbased5,Lunch_availability5,Session UUID_5_x,Device ID_6_x,Scheduled Time_6_x,Issued Time_6_x,Response Time_6_x,Duration (seconds) from scheduled to completion time_6_x,Duration (seconds) from first response to completion time_6_x,Location_6_x,Lunch_open6,Lunch_plantbased6,Lunch_availability6,Session UUID_6_x,Device ID_7_x,Scheduled Time_7_x,Issued Time_7_x,Response Time_7_x,Duration (seconds) from scheduled to completion time_7_x,Duration (seconds) from first response to completion time_7_x,Location_7_x,Lunch_open7,Lunch_plantbased7,Lunch_availability7,Session UUID_7_x,Device ID_8_x,Scheduled Time_8_x,Issued Time_8_x,Response Time_8_x,Duration (seconds) from scheduled to completion time_8_x,Duration (seconds) from first response to completion time_8_x,Location_8_x,Lunch_open8,Lunch_plantbased8,Lunch_availability8,Session UUID_8_x,Device ID_9_x,Scheduled Time_9_x,Issued Time_9_x,Response Time_9_x,Duration (seconds) from scheduled to completion time_9_x,Duration (seconds) from first response to completion time_9_x,Location_9_x,Lunch_open9,Lunch_plantbased9,Lunch_availability9,Session UUID_9_x,Device ID_10_x,Scheduled Time_10_x,Issued Time_10_x,Response Time_10_x,Duration (seconds) from scheduled to completion time_10_x,Duration (seconds) from first response to completion time_10_x,Location_10_x,Lunch_open10,Lunch_plantbased10,Lunch_availability10,Session UUID_10_x,Device ID_y,Scheduled Time_y,Issued Time_y,Response Time_y,Duration (seconds) from scheduled to completion time_y,Duration (seconds) from first response to completion time_y,Location_y,Message_liking1,Message_shareability1,Message_dislike_reason1,Message_relevance1_1,Message_relevance1_2,Message_relevance1_3,Session UUID_y,Device ID_2_y,Scheduled Time_2_y,Issued Time_2_y,Response Time_2_y,Duration (seconds) from scheduled to completion time_2_y,Duration (seconds) from first response to completion time_2_y,Location_2_y,Message_liking2,Message_shareability2,Message_dislike_reason2,Message_relevance2_1,Message_relevance2_2,Message_relevance2_3,Session UUID_2_y,Device ID_3_y,Scheduled Time_3_y,Issued Time_3_y,Response Time_3_y,Duration (seconds) from scheduled to completion time_3_y,Duration (seconds) from first response to completion time_3_y,Location_3_y,Message_liking3,Message_shareability3,Message_dislike_reason3,Message_relevance3_1,Message_relevance3_2,Message_relevance3_3,Session UUID_3_y,Device ID_4_y,Scheduled Time_4_y,Issued Time_4_y,Response Time_4_y,Duration (seconds) from scheduled to completion time_4_y,Duration (seconds) from first response to completion time_4_y,Location_4_y,Message_liking4,Message_shareability4,Message_dislike_reason4,Message_relevance4_1,Message_relevance4_2,Message_relevance4_3,Session 

In [82]:
#Lets save the combined daily surveys
#daily_surveys.to_csv("daily_surveys.csv")

### 4.5. Weekend surveys

In [64]:
#Lets load our three weekend surveys: baseline, t1, and t2

def clean_wave(df):
    # Step 1: Drop duplicate rows where Duration == "Expired"
    df = df[~((df.duplicated(subset='Name', keep=False)) & (df['Duration (seconds) from scheduled to completion time'] == "Expired"))]

    # Step 2: Keep only the last row for remaining duplicates
    df = df.drop_duplicates(subset='Name', keep='last')
    
    return df.reset_index(drop=True)

# Read and clean each file
w0 = clean_wave(pd.read_csv(WAVE5_DIR / "Weekend Survey 0.csv"))
w1 = clean_wave(pd.read_csv(WAVE5_DIR / "Weekend Survey 1.csv"))
w2 = clean_wave(pd.read_csv(WAVE5_DIR / "Weekend Survey 2.csv"))

#Well give each week file an identifier
w0["Week"] = 0
w1["Week"] = 1
w2["Week"] = 2

# Start with the first DataFrame
weekend_surveys = w0

# Merge the others one by one with suffixes to differentiate columns
for i, df in enumerate([w1, w2], start=1):
    suffix = f"_{i}"  # Adds suffix to the columns of each new dataframe
    weekend_surveys = pd.merge(weekend_surveys, df, on="Name", how="right", suffixes=('', suffix))

In [65]:
# Read and clean each file
w0_2 = clean_wave(pd.read_csv(WAVE6_DIR / "1. Baseline Survey.csv"))
w1_2 = clean_wave(pd.read_csv(WAVE6_DIR / "7. Weekend survey 1.csv"))
w2_2 = clean_wave(pd.read_csv(WAVE6_DIR / "13. Weekend survey 2.csv"))

w0_2["Week"] = 0
w2_2["Week"] = 1
w2_2["Week"] = 2

# Start with the first DataFrame
weekend_surveys_2 = w0_2

# Merge the others one by one with suffixes to differentiate columns
for i, df in enumerate([w1_2, w2_2], start=1):
    suffix = f"_{i}"  # Adds suffix to the columns of each new dataframe
    weekend_surveys_2 = pd.merge(weekend_surveys_2, df, on="Name", how="right", suffixes=('', suffix))

In [67]:
#Again we will need to filter by date to add the final wave entries

# Step 1: Remove trailing time zone strings (like " BST", " WAT", etc.)
weekend_surveys_2['Scheduled Time'] = (
    weekend_surveys_2['Scheduled Time']
    .astype(str)
    .str.replace(r'\s*([A-Z]{2,5}|[+-]?\d{2,4})$', '', regex=True)
)

print(len(weekend_surveys_2))

# Step 2: Convert to datetime
weekend_surveys_2['Scheduled Time'] = pd.to_datetime(
    weekend_surveys_2['Scheduled Time'], errors='coerce'
)

print(len(    weekend_surveys_2))

# Step 3: Drop timezone info (just in case)
if weekend_surveys_2['Scheduled Time'].dt.tz is not None:
    weekend_surveys_2['Scheduled Time'] = weekend_surveys_2['Scheduled Time'].dt.tz_localize(None)
print(len(    weekend_surveys_2))

# Step 4: Filter for May 1st or later
weekend_surveys_2 = weekend_surveys_2[
    weekend_surveys_2['Scheduled Time'] >= pd.Timestamp('2025-05-01')
]
print(len(    weekend_surveys_2))

# Step 5: Check result
print("Remaining rows after filter:", len(weekend_surveys_2))

673
673
673
42
Remaining rows after filter: 42


In [68]:
#Adding the final wave 6 entries
weekend_surveys = pd.concat([weekend_surveys, weekend_surveys_2])

In [71]:
#Lets compute some new variables by calculating their mean scores

# Define all your target variables and their components
averages = {
    "Sharing_contexts": ["Sharing_contexts1", "Sharing_contexts2", "Sharing_contexts3", "Sharing_contexts4"],
    "Perceived_control": ["Perceived_control1", "Perceived_control2", "Perceived_control3"],
    "Auto_eval": ["Aut_eval1", "Aut_eval2", "Aut_eval3", "Aut_eval4"],
    "Priv_self_eff_inst": ["Priv_self_eff_inst1", "Priv_self_eff_inst2"],
    "Priv_self_eff_int": ["Priv_self_eff_int1", "Priv_self_eff_int2", "Priv_self_eff_int3"],
    "Attitude_instrument1": ["Attitude_instrument1_1", "Attitude_instrument1_2"],
    "Attitude_experiential1": ["Attitude_experiential1_1", "Attitude_experiential1_2"],
    "Processing_motivation1": ["Processing_motivation1_1", "Processing_motivation1_2"],
    "Message_reactance1": ["Message_reactance1_1", "Message_reactance1_2", "Message_reactance1_3"],
    "Message_involvement1": ["Message_involvement1_1", "Message_involvement1_2", "Message_involvement1_3"],
    "Perceived_similarity1": ["Perceived_similarity1_1", "Perceived_similarity1_2"],
    "Source_liking1": ["Source_liking1_1", "Source_liking1_2"],
    "Source_trust1": ["Source_trust1_1", "Source_trust1_2", "Source_trust1_3", "Source_trust1_4"],
    "Attitude_instrument2": ["Attitude_instrument2_1", "Attitude_instrument2_2"],
    "Attitude_experiential2": ["Attitude_experiential2_1", "Attitude_experiential2_2"],
    "Processing_motivation2": ["Processing_motivation2_1", "Processing_motivation2_2"],
    "Message_reactance2": ["Message_reactance2_1", "Message_reactance2_2", "Message_reactance2_3"],
    "Message_involvement2": ["Message_involvement2_1", "Message_involvement2_2", "Message_involvement2_3"],
    "Perceived_similarity2": ["Perceived_similarity2_1", "Perceived_similarity2_2"],
    "Source_liking2": ["Source_liking2_1", "Source_liking2_2"],
    "Source_trust2": ["Source_trust2_1", "Source_trust2_2", "Source_trust2_3", "Source_trust2_4"]
}


def compute_composites(df: pd.DataFrame, scales: dict, warn_missing: bool = False) -> pd.DataFrame:

    for new_col, cols in scales.items():
        existing = [c for c in cols if c in df.columns]
        missing = [c for c in cols if c not in df.columns]

        if warn_missing and missing:
            print(f"Warning: missing columns for {new_col}: {missing}")

        if existing:
            df[new_col] = df[existing].mean(axis=1)
        else:
            df[new_col] = pd.NA  # no items present at all

    return df


# Compute composite mean scores for weekend survey scales
weekend_surveys = compute_composites(weekend_surveys, averages, warn_missing=True)


In [73]:
# Cronbach's Alpha for composite scales
def cronbach_alpha(df):
    df = df.dropna(axis=0)
    k = df.shape[1]
    if k < 2:
        return np.nan
    item_var = df.var(axis=0, ddof=1)
    total_var = df.sum(axis=1).var(ddof=1)
    return (k / (k - 1)) * (1 - item_var.sum() / total_var)

for scale, items in averages.items():
    existing = [c for c in items if c in weekend_surveys.columns]
    if len(existing) < 2:
        print(f"{scale}: not enough items → skipped")
        continue
    alpha = cronbach_alpha(weekend_surveys[existing])
    print(f"{scale}: α = {alpha:.2f}")


Sharing_contexts: α = 0.87
Perceived_control: α = 0.81
Auto_eval: α = 0.67
Priv_self_eff_inst: α = 0.63
Priv_self_eff_int: α = 0.81
Attitude_instrument1: α = 0.86
Attitude_experiential1: α = 0.81
Processing_motivation1: α = 0.86
Message_reactance1: α = 0.87
Message_involvement1: α = 0.81
Perceived_similarity1: α = 0.88
Source_liking1: α = 0.79
Source_trust1: α = 0.84
Attitude_instrument2: α = 0.86
Attitude_experiential2: α = 0.79
Processing_motivation2: α = 0.86
Message_reactance2: α = 0.85
Message_involvement2: α = 0.82
Perceived_similarity2: α = 0.89
Source_liking2: α = 0.77
Source_trust2: α = 0.86


In [74]:
# Renaming some column names
weekend_surveys.rename(columns={
    'Breakfast_plantbased1': 'Breakfast_plantbased',
    'Lunch_plantbased1': 'Lunch_plantbased',
    'Dinner_plantbased1': 'Dinner_plantbased',
    'Snacks_omnivore1': 'Snacks_omnivore',
    
    'Breakfast_plantbased2': 'Breakfast_plantbased1',
    'Lunch_plantbased2': 'Lunch_plantbased1',
    'Dinner_plantbased2': 'Dinner_plantbased1',
    'Snacks_omnivore2': 'Snacks_omnivore1',
    
    'Breakfast_plantbased3': 'Breakfast_plantbased2',
    'Lunch_plantbased3': 'Lunch_plantbased2',
    'Dinner_plantbased3': 'Dinner_plantbased2',
    'Snacks_omnivore3': 'Snacks_omnivore2',
}, inplace=True)

In [75]:
weekend_surveys.head(3)

,Name,Device ID,Scheduled Time,Issued Time,Response Time,Duration (seconds) from scheduled to completion time,Duration (seconds) from first response to completion time,Location,Breakfast_plantbased,Lunch_plantbased,Dinner_plantbased,Snacks_omnivore,Sharing_contexts1,Sharing_contexts2,Sharing_contexts3,Sharing_contexts4,Perceived_control1,Perceived_control2,Perceived_control3,Aut_eval1,Aut_eval2,Aut_eval3,Aut_eval4,Priv_self_eff_inst1,Priv_self_eff_inst2,Priv_self_eff_int1,Priv_self_eff_int2,Priv_self_eff_int3,Priv_Sev_int,Priv_sev_inst,Priv_sev_comm,Session UUID,Week,Device ID_1,Scheduled Time_1,Issued Time_1,Response Time_1,Duration (seconds) from scheduled to completion time_1,Duration (seconds) from first response to completion time_1,Location_1,Breakfast_plantbased1,Lunch_plantbased1,Dinner_plantbased1,Snacks_omnivore1,Attitude_instrument1_1,Attitude_instrument1_2,Attitude_experiential1_1,Attitude_experiential1_2,Intention_plantbased1,Message_personalized1,Message_similarity1,Processing_motivation1_1,Processing_motivation1_2,Message_reactance1_1,Message_reactance1_2,Message_reactance1_3,Message_involvement1_1,Message_involvement1_2,Message_involvement1_3,Perceived_similarity1_1,Perceived_similarity1_2,Source_liking1_1,Source_liking1_2,Source_trust1_1,Source_trust1_2,Source_trust1_3,Source_trust1_4,Session UUID_1,Week_1,Device ID_2,Scheduled Time_2,Issued Time_2,Response Time_2,Duration (seconds) from scheduled to completion time_2,Duration (seconds) from first response to completion time_2,Location_2,Breakfast_plantbased2,Lunch_plantbased2,Dinner_plantbased2,Snacks_omnivore2,Attitude_instrument2_1,Attitude_instrument2_2,Attitude_experiential2_1,Attitude_experiential2_2,Intention_plantbased2,Message_personalized2,Message_similarity2,Processing_motivation2_1,Processing_motivation2_2,Message_reactance2_1,Message_reactance2_2,Message_reactance2_3,Message_involvement2_1,Message_involvement2_2,Message_involvement2_3,Perceived_similarity2_1,Perceived_similarity2_2,Source_liking2_1,Source_liking2_2,Source_trust2_1,Source_trust2_2,Source_trust2_3,Source_trust2_4,Program_pleasure,Program_improvement,Program_challenge,AI_awareness,Tailroing_attitude1,Tailoring_attitude2,Program_feedback,[77_SAQ] <p><strong>We are organizing a follow-up online vi,[78_FFT] <p><strong>Thank you for your interest in the vide,Session UUID_2,Week_2,Sharing_contexts,Perceived_control,Auto_eval,Priv_self_eff_inst,Priv_self_eff_int,Attitude_instrument1,Attitude_experiential1,Processing_motivation1,Message_reactance1,Message_involvement1,Perceived_similarity1,Source_liking1,Source_trust1,Attitude_instrument2,Attitude_experiential2,Processing_motivation2,Message_reactance2,Message_involvement2,Perceived_similarity2,Source_liking2,Source_trust2
0,111562,dfb2946c4d7e8afa,2025-02-01 09:00:00 GMT,2025-02-01 09:00:00 GMT,2025-02-01 11:57:22 GMT,10642.282,152,Unknown,4.0,3.0,5.0,4.0,2.0,2.0,2.0,2.0,5.0,5.0,6.0,5.0,6.0,4.0,5.0,4.0,5.0,6.0,5.0,6.0,5.0,5.0,6.0,87197f9e-5abe-4f70-9c9b-84c4a0def7ed,0.0,dfb2946c4d7e8afa,2025-02-08 09:00:00 GMT,2025-02-08 09:00:00 GMT,2025-02-08 16:05:17 GMT,25517.858,25518,Unknown,2.0,3.0,3.0,3.0,3.0,3.0,4.0,3.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,b36b518c-0f82-43ed-9658-4c24d076c4d8,1.0,dfb2946c4d7e8afa,2025-02-15 09:00:00 GMT,2025-02-15 09:00:00 GMT,2025-02-15 17:36:01 GMT,30961.037,30961,Unknown,3.0,4.0,2.0,3.0,4.0,4.0,4.0,4.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.0,6.0,2.0,2.0,5.0,It was naturally flowing and I loved it,None,1.0,Merrynet76@gmail.com,f6b18624-3f31-4a2c-8c3a-9d67763d133f,2,2.0,5.333333,5.0,4.5,5.666667,3.0,3.5,NaN,NaN,NaN,NaN,NaN,NaN,4.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN
1,111584,8ca75ff822855263,2025-02-01 09:00:00 GMT,2025-02-01 09:00:00 GMT,2025-02-01 09:05:17 GMT,317.565,120,Unknown,3.0,1.0,2.0,1.0,2.0,2.0,1.0,1.0,5.0,5.0,6.0,5.0,6.0,5.0,6.0,6.0,6.0,6.0,6.0,5.0,5.0,2.0,3.0,c42a2716-c5f7-4439-8acc-fc3c2c115dee,0.0,e1386515408cdfdc,2025-02-08 09:

## 5. Data Transformation

### 5. Weekend surveys: Transforming from wide to long format

In [76]:
df = weekend_surveys.copy()

id_col = "Name"

# Map week index to suffix used in column names
week_suffix = {0: "", 1: "1", 2: "2"}

base_vars = [
    "Breakfast_plantbased", "Lunch_plantbased", "Dinner_plantbased", "Snacks_omnivore",
    "Attitude_instrument", "Attitude_experiential", "Processing_motivation",
    "Message_reactance", "Message_involvement", "Perceived_similarity",
    "Source_liking", "Source_trust", "Intention_plantbased",
    "Message_personalized", "Message_similarity",
]

week0_only = [
    "Sharing_contexts", "Perceived_control", "Auto_eval",
    "Priv_self_eff_inst", "Priv_self_eff_int",
]

week2_only = [
    "Program_pleasure", "Program_improvement", "Program_challenge",
    "AI_awareness", "Tailroing_attitude1", "Tailoring_attitude2", "Program_feedback",
]

long_dfs = []

for week, suffix in week_suffix.items():
    week_df = pd.DataFrame({
        id_col: df[id_col],
        "Week": week,
    })

    # Repeated variables (week-specific suffix where applicable)
    for base_name in base_vars:
        col_name = f"{base_name}{suffix}" if suffix else base_name
        week_df[base_name] = df[col_name] if col_name in df.columns else pd.NA

    # Week 0-only variables
    for col in week0_only:
        week_df[col] = df[col] if week == 0 and col in df.columns else pd.NA

    # Week 2-only variables
    for col in week2_only:
        week_df[col] = df[col] if week == 2 and col in df.columns else pd.NA

    long_dfs.append(week_df)

weekend_surveys_long = pd.concat(long_dfs, ignore_index=True)

print(len(df), "rows in wide format →", len(weekend_surveys_long), "rows in long format")


674 rows in wide format → 2022 rows in long format


C:\Users\77197jsc\AppData\Local\Temp\ipykernel_9792\1120424604.py:49: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  weekend_surveys_long = pd.concat(long_dfs, ignore_index=True)


In [77]:
weekend_surveys_long.head()

,Name,Week,Breakfast_plantbased,Lunch_plantbased,Dinner_plantbased,Snacks_omnivore,Attitude_instrument,Attitude_experiential,Processing_motivation,Message_reactance,Message_involvement,Perceived_similarity,Source_liking,Source_trust,Intention_plantbased,Message_personalized,Message_similarity,Sharing_contexts,Perceived_control,Auto_eval,Priv_self_eff_inst,Priv_self_eff_int,Program_pleasure,Program_improvement,Program_challenge,AI_awareness,Tailroing_attitude1,Tailoring_attitude2,Program_feedback
0,111562,0,4.0,3.0,5.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.00,5.333333,5.00,4.5,5.666667,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,111584,0,3.0,1.0,2.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.50,5.333333,5.50,6.0,5.666667,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,111598,0,3.0,3.0,2.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.50,4.333333,4.50,4.5,5.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,111602,0,1.0,1.0,3.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.25,3.000000,3.75,4.0,4.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,111605,0,3.0,5.0,3.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.25,4.666667,4.25,4.0,5.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [78]:
weekend_surveys_long = weekend_surveys_long.sort_values(by='Name').reset_index(drop=True)

In [80]:
# Load IDs that meet minimum completion criteria
half_completes2 = pd.read_csv(DATA_DIR / "processed" / "half_complete_IDs2.csv")

# Ensure numeric match for safety
valid_ids = pd.to_numeric(half_completes2["User ID"], errors="coerce")

# Filter weekend long data
weekend_surveys_filtered = (
    weekend_surveys_long[
        pd.to_numeric(weekend_surveys_long["Name"], errors="coerce").isin(valid_ids)
    ]
    .reset_index(drop=True)
)

print("Filtered rows:", len(weekend_surveys_filtered))


Filtered rows: 1821


In [83]:
# Drop duplicate Name–Week combos and remove rows that are all-NaN except ID + Week
cols_to_check = weekend_surveys_filtered.columns.difference(["Name", "Week"])

weekend_surveys_filtered_clean = (
    weekend_surveys_filtered
    .drop_duplicates(subset=["Name", "Week"])
    .dropna(subset=cols_to_check, how="all")
    .reset_index(drop=True)
)

print("Rows after cleaning:", len(weekend_surveys_filtered_clean))

# Check IDs with more than 3 entries
name_counts = pd.to_numeric(weekend_surveys_filtered_clean["Name"], errors="coerce").value_counts()
print("Names with >3 rows:\n", name_counts[name_counts > 3])


Rows after cleaning: 1814
Names with >3 rows:
 Series([], Name: count, dtype: int64)


In [84]:
#Again lets merge with the keys file
weekend_surveys_filtered_clean = weekend_surveys_filtered_clean.merge(keys_all[["User ID", "condition"]], left_on = "Name", right_on = "User ID")

#Merging with Qualtrics
weekend_surveys_filtered_clean = weekend_surveys_filtered_clean.merge(qualtrics_data, how = "left", left_on = "Name", right_on = "User ID")

#And manually correct one mistake
weekend_surveys_filtered_clean.loc[
    weekend_surveys_filtered_clean["Name"] == 108518, "condition"
] = 2

In [85]:
weekend_surveys_filtered_clean.head(5)

,Name,Week,Breakfast_plantbased,Lunch_plantbased,Dinner_plantbased,Snacks_omnivore,Attitude_instrument,Attitude_experiential,Processing_motivation,Message_reactance,Message_involvement,Perceived_similarity,Source_liking,Source_trust,Intention_plantbased,Message_personalized,Message_similarity,Sharing_contexts,Perceived_control,Auto_eval,Priv_self_eff_inst,Priv_self_eff_int,Program_pleasure,Program_improvement,Program_challenge,AI_awareness,Tailroing_attitude1,Tailoring_attitude2,Program_feedback,User ID_x,condition_x,Unnamed: 0.1,Unnamed: 0,User ID_y,Baseline,Day 1,Day 2,Day 3,Day 4,Day 5,Weekend 1,Day 6,Day 7,Day 8,Day 9,Day 10,Weekend 2,Norstat ID_x,condition_y,ID,Movez ID,Norstat ID,Message_exposure,Mail_child_exp,Mail_child_ctr,Country_of_residence,Country_of_residence_6_TEXT,Dietary_identity,Dietary_identity_6_TEXT,Children_age_years,Birthplace_parents,Ethnic_group,Ethnic_group_5_TEXT,SES_education,SES_income_UK,SES_income_ROI,Random ID,id,SES,Child_ID,Mail_child,Imposter,Game_pleasure,Game_relatedness,Game_capability,Sharing_behavior_gam,Game_rating,Game_feedback,FFQ_meat,FFQ_dairy,FFQ_eggs,FFQ_fish,Gender,Gender_11_TEXT,FFQ_average,condition
0,104582,0,3.0,2.0,3.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,5.666667,5.5,6.0,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,104582.0,2.0,0,0,104582.0,1.0,1,1,1,1,1,1.0,1,1,1,1,1,1.0,5nJ4ApbP4Ne2G3kx,2.0,104582.0,43876.0,NaN,1.0,romeobenson23@gmail.com,NaN,1,NaN,1,NaN,5,2,NaN,NaN,5.0,5.0,NaN,43876.0,5nJ4ApbP4Ne2G3kx,5.0,43876,romeobenson23@gmail.com,NaN,6.0,6.0,6.0,3,5.0,NaN,5.0,5.0,5.0,5.0,7,NaN,5.0,NaN
1,104582,2,3.0,5.0,3.0,3.0,4.0,4.0,6.0,1.333333,6.000000,4.5,4.0,4.0,4.0,6.0,6.0,NaN,NaN,NaN,NaN,NaN,6.0,6.0,5.0,1.0,5.0,I like this communication style matching becau...,nothing more.,104582.0,2.0,0,0,104582.0,1.0,1,1,1,1,1,1.0,1,1,1,1,1,1.0,5nJ4ApbP4Ne2G3kx,2.0,104582.0,43876.0,NaN,1.0,romeobenson23@gmail.com,NaN,1,NaN,1,NaN,5,2,NaN,NaN,5.0,5.0,NaN,43876.0,5nJ4ApbP4Ne2G3kx,5.0,43876,romeobenson23@gmail.com,NaN,6.0,6.0,6.0,3,5.0,NaN,5.0,5.0,5.0,5.0,7,NaN,5.0,NaN
2,104582,1,4.0,4.0,3.0,1.0,4.0,4.0,6.0,1.666667,5.666667,4.0,4.0,4.0,5.0,6.0,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,104582.0,2.0,0,0,104582.0,1.0,1,1,1,1,1,1.0,1,1,1,1,1,1.0,5nJ4ApbP4Ne2G3kx,2.0,104582.0,43876.0,NaN,1.0,romeobenson23@gmail.com,NaN,1,NaN,1,NaN,5,2,NaN,NaN,5.0,5.0,NaN,43876.0,5nJ4ApbP4Ne2G3kx,5.0,43876,romeobenson23@gmail.com,NaN,6.0,6.0,6.0,3,5.0,NaN,5.0,5.0,5.0,5.0,7,NaN,5.0,NaN
3,104584,0,1.0,1.0,1.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.5,6.000000,5.0,6.0,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,104584.0,3.0,1,1,104584.0,1.0,1,1,1,1,1,1.0,1,1,1,1,1,1.0,Vw142r07XqPAaNPX,3.0,104584.0,64938.0,NaN,1.0,gaagoobilo@gmail.com,NaN,1,NaN,1,NaN,3,2,NaN,NaN,4.0,4.0,NaN,64938.0,Vw142r07XqPAaNPX,4.0,64938,gaagoobilo@gmail.com,NaN,6.0,6.0,6.0,6,5.0,I liked the simple user interface. I disliked ...,4.0,4.0,4.0,4.0,7,NaN,4.0,NaN
4,104584,2,6.0,5.0,5.0,3.0,4.0,4.0,6.0,1.000000,6.000000,5.5,4.0,4.0,4.0,6.0,5.0,NaN,NaN,NaN,NaN,NaN,6.0,6.0,4.0,1.0,6.0,I like the idea of the communication style mat...,I liked the daily messages they were informati...,104584.0,3.0,1,1,104584.0,1.0,1,1,1,1,1,1.0,1,1,1,1,1,1.0,Vw142r07XqPAaNPX,3.0,104584.0,64938.0,NaN,1.0,gaagoobilo@gmail.com,NaN,1,NaN,1,NaN,3,2,NaN,NaN,4.0,4.0,NaN,64938.0,Vw142r07XqPAaNPX,4.0,64938,gaagoobilo@gmail.com,NaN,6.0,6.0,6.0,6,5.0,I liked the simple user interface. I disliked ...,4.0,4.0,4.0,4.0,7,NaN,4.0,NaN


In [86]:
# Filter rows where Country_of_residence is not NaN and Children_age_years is NaN
filtered_df = weekend_surveys_filtered_clean[
    weekend_surveys_filtered_clean["Country_of_residence"].notna() &
    weekend_surveys_filtered_clean["Children_age_years"].isna()
]

# Count the number of such instances
count = len(filtered_df)

# Display the count
print("Number of instances:", count)

Number of instances: 0


In [87]:
weekend_surveys_filtered_clean.to_csv(DATA_DIR / "processed" / "weekend_surveys.csv")

### 6. Daily Surveys: From wide to long format

In [88]:
# Deduplicate daily survey rows by keeping the one with the most non-NaN values
daily_surveys = (
    daily_surveys
    .assign(non_na_count=lambda df: df.notna().sum(axis=1))
    .sort_values(["Name", "non_na_count"], ascending=[True, False])
    .drop_duplicates(subset="Name", keep="first")
    .drop(columns="non_na_count")
    .reset_index(drop=True)
)

print("Remaining duplicate Names:", daily_surveys["Name"].duplicated().sum())


Remaining duplicate Names: 0


In [89]:
#Now we can transofrm our dataframe fram wide format into long

# Set index so wide_to_long works
daily_surveys_long = pd.wide_to_long(
    daily_surveys,
    stubnames=['Message_liking', 'Message_shareability', 'Message_dislike_reason', 'Message_relevance', "Lunch_open", "Lunch_plantbased", "Lunch_availability"],
    i='Name',
    j='Day',
    sep='',
    suffix='\\d+'
).reset_index()

In [90]:
# Lets only keep columns we are interested in, and also create a new week variabble

daily_surveys_long = daily_surveys_long[["Name", "Day", "Message_liking","Message_shareability", "Message_dislike_reason","Message_relevance", "Lunch_open", "Lunch_plantbased", "Lunch_availability"]]
daily_surveys_long['week'] = daily_surveys_long['Day'].apply(lambda x: 1 if x <= 5 else 2)

In [776]:
#morning_surveys_long = morning_surveys_long[morning_surveys_long["Message_liking"].notna()]

In [91]:
#Lets sort our values..
daily_surveys_long = daily_surveys_long.sort_values(by=['Name', 'Day'])
daily_surveys_long.head()

,Name,Day,Message_liking,Message_shareability,Message_dislike_reason,Message_relevance,Lunch_open,Lunch_plantbased,Lunch_availability,week
0,96792,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
682,96792,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
1364,96792,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
2046,96792,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
2728,96792,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1


In [119]:
#Lets see have this data would look like for one user with 7 valid survey entries
daily_surveys_long[daily_surveys_long["Name"] == 108626]

,Name,Day,Message_liking,Message_shareability,Message_dislike_reason,Message_relevance,Lunch_open,Lunch_plantbased,Lunch_availability,week
143,108626.0,1,NaN,NaN,NaN,NaN,ham and cheese sandwich,1.0,2.0,1
825,108626.0,2,NaN,NaN,NaN,NaN,ham and cheese sandwich,2.0,1.0,1
1507,108626.0,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
2189,108626.0,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
2871,108626.0,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
3553,108626.0,6,2.0,1.0,NaN,4.333333,snack bars,3.0,1.0,2
4235,108626.0,7,2.0,2.0,NaN,5.000000,a variety of sandwiches,1.0,1.0,2
4917,108626.0,8,2.0,2.0,NaN,5.666667,ham and cheese sandwich,1.0,1.0,2
5599,108626.0,9,2.0,2.0,NaN,5.333333,ham and cheese sandwich,1.0,1.0,2
6281,108626.0,10,2.0,2.0,NaN,5.333333,snack bar,3.0,1.0,2


In [120]:
#Lets ahve a first sneak peak on adoleescents lunch choices
daily_surveys_long["Lunch_plantbased"].value_counts()

Lunch_plantbased
1.0    2884
2.0    1981
3.0    1101
4.0     114
Name: count, dtype: int64

In [92]:
# To get the condition we merge with the keys file...
daily_surveys_long["Name"] = daily_surveys_long["Name"].astype(float)
daily_surveys_long_keys = daily_surveys_long.merge(keys_all[['User ID', 'condition']], how = "left", left_on= "Name", right_on = "User ID")

In [93]:
#And merge everything together
daily_surveys_long_all = daily_surveys_long_keys.merge(qualtrics_data, how = "left", left_on= "Name", right_on = "User ID")
daily_surveys_long_all["condition_x"] = daily_surveys_long_all["condition_x"].fillna(daily_surveys_long_all["condition_y"])

In [94]:
# Identify unmatched rows (no condition assigned)
unmatched_mask = daily_surveys_long_all["condition_x"].isna()
unmatched_rows = daily_surveys_long_all[unmatched_mask]

unmatched_count = unmatched_mask.sum()
print(f"{unmatched_count} entries could not be matched to a condition.")

# Show unique IDs that failed to match
print("Unmatched Names:", unmatched_rows["Name"].unique())


90 entries could not be matched to a condition.
Unmatched Names: [ 96792.  97152. 104553. 110202. 111509. 111709. 111768. 115168. 115681.]


In [97]:
# IDs to manually correct
ids_to_set = [111509, 111709, 111768, 115168, 115681]

# Assign condition = 1 for those IDs
mask = daily_surveys_long_all["Name"].isin(ids_to_set)
daily_surveys_long_all.loc[mask, "condition_x"] = 1

print(f"Manually corrected {mask.sum()} entries.")

# Remove remaining rows with missing condition
daily_surveys_long_all = daily_surveys_long_all.dropna(subset=["condition_x"]).reset_index(drop=True)

print("Remaining missing conditions:", daily_surveys_long_all["condition_x"].isna().sum())


Manually corrected 50 entries.
Remaining missing conditions: 0


### 7. Filtering for Completes Survey Resposnes

In [99]:
# As last filtering step we only want to keep IDs that fulffilled our minimum standards for complete survey response (at lest 4 daily surveys, all three weekend surveys). IDs that fulffill these criteria we can read from the following file:

half_complete_days = pd.read_csv(DATA_DIR / "processed" / "half_complete_IDs.csv") #very strict criteria
half_complete_days2 = pd.read_csv(DATA_DIR / "processed" / "half_complete_IDs2.csv") #normal criteria

print(len(half_complete_days))
print(len(half_complete_days2))

589
622


In [100]:
# Ensure numeric IDs once
ids_all = pd.to_numeric(daily_surveys_long_all["Name"], errors="coerce")
ids_req1 = pd.to_numeric(half_complete_days["User ID"], errors="coerce")
ids_req2 = pd.to_numeric(half_complete_days2["User ID"], errors="coerce")

# Filter datasets
daily_surveys_filtered = daily_surveys_long_all[ids_all.isin(ids_req1)].reset_index(drop=True)
daily_surveys_filtered2 = daily_surveys_long_all[ids_all.isin(ids_req2)].reset_index(drop=True)

print("Filtered (set 1):", len(daily_surveys_filtered))
print("Filtered (set 2):", len(daily_surveys_filtered2))


Filtered (set 1): 5860
Filtered (set 2): 6130


In [101]:
# Lets check in reverse if there are any users that did complete the surveys, but are not within our combined dataframe

# Find the User IDs in half_complete_days that are NOT in daily_surveys_long
missing_ids = half_complete_days[~half_complete_days["User ID"].isin(daily_surveys_long_all["Name"])]

# Count the missing IDs
missing_ids_count = missing_ids.shape[0]

# Print the missing IDs and the count
print(f"Missing IDs count: {missing_ids_count}")
print(missing_ids["User ID"])

Missing IDs count: 3
91     108385
138    108772
176    111495
Name: User ID, dtype: int64


In [103]:
daily_surveys_filtered[
    ["Message_liking", "Message_shareability", "Message_dislike_reason",
     "Message_relevance", "Lunch_plantbased", "Lunch_availability"]
].mean(numeric_only=True)


Message_liking            2.562720
Message_shareability      2.487322
Message_dislike_reason    1.374517
Message_relevance         5.798846
Lunch_plantbased          1.758992
Lunch_availability        1.361694
dtype: float64

### 8. Data Imputation for Missing Values (*Note: not used in the final analysis*)

In [104]:
# First, sort by Name and Day
daily_surveys_filtered3 = daily_surveys_filtered2.sort_values(by=['Name', 'Day'])

# Then define a function to impute per participant backward, given that past behavior is the best predictor of future behavior
def backward_impute(group):
    # We sort again to be safe
    group = group.sort_values('Day')
    # Fill missing values by looking backward
    group['Lunch_plantbased'] = group['Lunch_plantbased'].ffill()
    return group

# Apply this per Name
daily_surveys_long_imputed = daily_surveys_filtered3.groupby('Name').apply(backward_impute).reset_index(drop=True)

C:\Users\77197jsc\AppData\Local\Temp\ipykernel_9792\2434842530.py:13: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  daily_surveys_long_imputed = daily_surveys_filtered3.groupby('Name').apply(backward_impute).reset_index(drop=True)


In [105]:
daily_surveys_long_imputed["Lunch_plantbased"].isna().sum()
# Lets remove the NA values in condition
daily_surveys_long_imputed = daily_surveys_long_imputed[pd.notna(daily_surveys_long_imputed["condition_x"])]

### 9. Final Data Clearning and Datafiles

In [106]:
# Recoding maps
lunch_map = {4: 0}
availability_map = {2: 1, 3: 2, 1: 3}

# List of all DataFrames that need these recodings
dfs = [
    daily_surveys_filtered,
    daily_surveys_filtered2,
    daily_surveys_long,
    daily_surveys_long_imputed,
    daily_surveys_long_all
]

# Apply recodings
for df in dfs:
    df["Lunch_plantbased"] = df["Lunch_plantbased"].replace(lunch_map)
    df["Lunch_availability"] = df["Lunch_availability"].replace(availability_map)


In [107]:
# We can save the final dataframe now

daily_surveys_filtered.to_csv(DATA_DIR / "processed" / "daily_surveys_filtered.csv") #strict filter
daily_surveys_filtered2.to_csv(DATA_DIR / "processed" /"daily_surveys_filtered2.csv") #normal filter
daily_surveys_long_all.to_csv(DATA_DIR / "processed" /"daily_surveys_long.csv") #non-imputed non-filtered data
daily_surveys_long_imputed.to_csv(DATA_DIR / "processed" /"daily_surveys_long_imputed.csv") #imputed datafile

print("Strict filter df:", len(daily_surveys_filtered))
print("Normal filter df:",len(daily_surveys_filtered2))
print("Nonfiltered filter df:",len(daily_surveys_long))
print("Imputed data df:",len(daily_surveys_long_imputed))

Strict filter df: 5860
Normal filter df: 6130
Nonfiltered filter df: 6820
Imputed data df: 6130
